*Ceci est le classeur numéro **4** de la matière Langage C du **2nd semestre**. Il s'agit d'un classeur **optionnel**, destiné à ceux qui veulent aller un peu plus loin dans l'utilisation du C.*

<br>

**Ne consultez ce classeur que si vous avez fini les 3 précédents et si vous êtes à l'aise avec le C.**

Le classeur propose par ailleurs une modification du projet pour rendre l'application un peu plus intéractive, à ne réaliser que si votre projet est déjà fonctionnel !

<br>

Ce classeur aborde le point suivants :
 - pointeur de fonction


<center><img src="dijkstra.gif" alt="Dijkstra animé" width="70%" /></center>


## Les pointeurs de fonction

La philosophie du C est une application très stricte du principe de l'[architecture von Neumann](https://en.wikipedia.org/wiki/Von_Neumann_architecture) : le code est des données autant que les données.

Très concrètement, en C, une fonction correspond à un bloc dans la mémoire. Bien sûr, les fonctions ne sont pas dans la même zone que les autres variables, mais elles sont tout de même quelque part dans la mémoire vive : les fonctions ont des adresses, et sont donc référençables à loisir !

Prenons un exemple très concret :


In [1]:
//%cflags: -O0
#include <stdlib.h>
#include <stdio.h>

// on utilise l'attribut noinline de GCC pour l'empêcher d'inliner la fonction
void __attribute__ ((noinline)) f1(int a) {
    printf("@f1.a = %p\n", &a);
    printf("%d\n", a);
}

int __attribute__ ((noinline)) f2(int b) {
    printf("@f2.b = %p\n", &b);
    return b + 1;
}

int main() {
    int x = 10;
    int y = 11;
    f1(x);
    int z = f2(y);
    printf("%d\n", z);

    puts("==========");
    printf("@x = %p\n", &x);
    printf("@y = %p\n", &y);
    printf("@z = %p\n", &z);
    printf("@f1 = %p\n", &f1); // Les fonctions ont une adresse aussi !
    printf("@f2 = %p\n", &f2);
    printf("@main = %p\n", &main);

    return 0;
}

@f1.a = 0x7ffef2a39dac
10
@f2.b = 0x7ffef2a39dac
12
@x = 0x7ffef2a39dcc
@y = 0x7ffef2a39dd0
@z = 0x7ffef2a39dd4
@f1 = 0x7b65da6b1179
@f2 = 0x7b65da6b11bf
@main = 0x5bb69b71e1c9


La conséquence directe de ça, c'est que les fonctions sont effectivement des _citoyens de première classe_ : on peut les stocker dans des variables, et les passer en paramètres d'autres fonctions ! Attention cependant, cela doit forcément passer par un _pointeur_, pour des raisons techniques.

Une question se pose alors : quel est le type d'un pointeur sur une fonction ? Ce type dépend en fait du prototype de la fonction, et a la forme générale suivante :

```c
<type retour>(*<nom de variable>)(<type paramètre 1>, <type paramètre 2>, ...)
```

Attention, le type est bien en trois parties, et le nom de la variable (ou du paramètre) qui reçoit la fonction est "au milieu" du type :
 1. Le type de retour
 2. L'étoile (suivie du nom de la variable)
 3. Le type des paramètres

Par exemple, si je veux stocker la fonction de prototype `int fun(char x, double y)` dans la variable `f` alors je dois déclarer `f` de la manière suivante :

```c
int(*f)(char,double);
```

Ensuite, une fois qu'une variable-fonction est définie, c'est factuellement une fonction pour C : on peut l'appeler en faisant `f(<argument 1>, <argument 2>, ...)` sans problème !

<br>

Attention : dès qu'il s'agit de pointeurs de fonction, le compilateur C est très laxiste sur le contrôle de type... Il accepte en particulier des fonctions qui n'ont pas le bon type de paramètre (il détecte un peu mieux les incompatibilité de type de retour, même si ce n'est pas toujours le cas).

<br>

Voyons quelques exemples avec des types de fonction différents :


In [4]:
//%cflags: -O0 -Wall -Wextra
#include <stdlib.h>
#include <stdio.h>

void p1() {
    puts("Fonction p1");
}

void p2() {
    puts("Fonction p2");
}

void p3(int x) {
    printf("Fonction p3 (x = %d)\n", x);
}

int p4() {
    puts("Fonction p4");
    return 1;
}

int main() {
    void(*f)(); // Le type des fonctions qui retournent void et n'ont pas de paramètre

    f = &p1; // p1 est compatible
    f();

    f = &p2;
    f("abc", 153); // GCC s'en fiche :| (parce que f ne déclare pas de paramètre)

    f = &p3; // même pas de warning...
    f();
    f(151);  // ???? et en plus ça l'affiche bien

    f = &p4; // un warning à cause du type de retour
    f();
    // int r = f(); // C'est illégal car f retourne un void formellement, la valeur de retour est perdue :(

    return 0;
}

/tmp/tmpayr8d5yf.c: In function ‘main’:
/tmp/tmpayr8d5yf.c:35:7: warning: assignment to ‘void (*)()’ from incompatible pointer type ‘int (*)()’ [-Wincompatible-pointer-types]
   35 |     f = &p4; // un warning à cause du type de retour
      |       ^


Fonction p1
Fonction p2
Fonction p3 (x = -297678224)
Fonction p3 (x = 151)
Fonction p4


Dans ce premier exemple, on constate que GCC ne fait pas beaucoup d'efforts pour vérifier la cohérence des types entre fonctions. Cela crée des phénomènes étranges : **si un pointeur de fonction n'indique pas de paramètres, GCC comprend en fait qu'on peut y mettre autant d'argument qu'on veut** (oui).

Si on ne met aucun argument et que la fonction utilise un de ses paramètres, c'est comme si on avait une variable déclarée mais non définie, donc la valeur n'est pas définie (donc globalement non-déterministe).


In [1]:
//%cflags: -O0 -Wall -Wextra
#include <stdlib.h>
#include <stdio.h>

int f1(char x, double y) {
    printf("f1 : %c\n", x);
    return ((int) y) * 2;
}

int f2(char x, double y) {
    printf("f2 : %c\n", x);
    return (int)(y * 2.0);
}

int f3(char x, char y) {
    printf("f3 : %c, %c\n", x, y);
    return 1;
}

int f4(char x) {
    printf("f4 : %c\n", x);
    return 0;
}

void f5(char x, double y) {
    printf("f5 : %c, %f\n", x, y);
}

int main() {
    int(*f)(char,double);
    int resultat;

    f = &f1;
    resultat = f('a', 3.7);
    printf("resultat = %d\n", resultat);

    f = &f2;
    resultat = f('a', 3.7);
    printf("resultat = %d\n", resultat);

    f = &f3; // Un warning car les paramètres ne correspondent pas
    resultat = f('a', 'b'); // L'appel ne se passe pas bien car C empile un char comme si c'était un double avant d'appeler f3
    printf("resultat = %d\n", resultat);
    resultat = f('a', 3.7); // La fonction dépile un char mais un double a été empilé... ça se passe également très mal
    printf("resultat = %d\n", resultat);

    f = &f4; // Un warning car pas assez de paramètres
    resultat = f('x', 1.0);
    // resultat = f('x');   // Illégal ! le type de f impose pas moins de deux arguments !
    // resultat = f('x', 1.0, "abc"); // Illégal ! le type de f impose pas plus de deux arguments !
    printf("resultat = %d\n", resultat);

    f = &f5; // Un warning car pas le bon type de retour
    resultat = f('z', 3.7); // Hmm mais f5 ne retourne rien
    printf("resultat = %d\n", resultat); // Du coup la valeur de resultat n'est pas déterminée...

    return 0;
}

/tmp/tmp1feo4vj7.c: In function ‘main’:
/tmp/tmp1feo4vj7.c:41:7: warning: assignment to ‘int (*)(char,  double)’ from incompatible pointer type ‘int (*)(char,  char)’ [-Wincompatible-pointer-types]
   41 |     f = &f3; // Un warning car les paramètres ne correspondent pas
      |       ^
/tmp/tmp1feo4vj7.c:47:7: warning: assignment to ‘int (*)(char,  double)’ from incompatible pointer type ‘int (*)(char)’ [-Wincompatible-pointer-types]
   47 |     f = &f4; // Un warning car pas assez de paramètres
      |       ^
/tmp/tmp1feo4vj7.c:53:7: warning: assignment to ‘int (*)(char,  double)’ from incompatible pointer type ‘void (*)(char,  double)’ [-Wincompatible-pointer-types]
   53 |     f = &f5; // Un warning car pas le bon type de retour
      |       ^


Le compilateur vérifie que les fonctions sont compatibles (mais si ce n'est pas le cas ce n'est _que_ un warning...).

Par contre lorsque le type déclare des paramètres, le compilateur vérifie qu'il y a bien le bon nombre de paramètres à l'appel de fonction, comme si c'était une fonction normale (et si ce n'est pas le cas c'est bien une erreur).

En fait, pour forcer le compilateur à exclure les appels de fonction avec des arguments alors que la fonction n'en définit pas, il faut dire explicitement qu'il n'y a pas de paramètre, en mettant `void` dans les parenthèses.

Cela vient d'une vieille syntaxe de C... Nous ne rentrerons pas trop dans les détails. Pour l'heure, rappelez vous que pour éviter les erreurs au maximum, on met `void` entre les parenthèses de la signature de fonction si cette dernière ne définit pas.


In [1]:
//%cflags: -O0 -Wall -Wextra
#include <stdlib.h>
#include <stdio.h>

void p1() {
    puts("p1");
}

void p2(void) {
    puts("p2");
}

void p3(int x) {
    printf("p3(%d)\n", x);
}

int main() {
    void(*f)(void);

    f = &p1;
    f();

    f = &p2;
    f();

    f = &p3;
    f();
    // f(51); // GCC détecte bien que ce n'est pas possible ! (ouf)

    return 0;
}

/tmp/tmp_jymbfrc.c: In function ‘main’:
/tmp/tmp_jymbfrc.c:26:7: warning: assignment to ‘void (*)(void)’ from incompatible pointer type ‘void (*)(int)’ [-Wincompatible-pointer-types]
   26 |     f = &p3;
      |       ^


p1
p2
p3(834783856)


### Fonctions en paramètre

L'intérêt principal des pointeurs de fonction est de pouvoir passer des fonctions en paramètre d'autres fonctions, ce qui permet typiquement d'écrire des fonctions un peu plus "génériques" (des fonctions de parcours par exemple).

C'est également une forme d'[_inversion de contrôle_](https://en.wikipedia.org/wiki/Inversion_of_control), qui est au coeur de diverses pratiques communes en informatique (programmation événementielle, exceptions, etc.).

Par exemple, définissons rapidement des traitements élément-à-élément sur des tableaux :


In [2]:
#include <stdio.h>
#include <stdlib.h>

struct vec_t {
    size_t size;
    float* content;
};
typedef struct vec_t vec_t;

vec_t creer(const size_t size, const float* tab) {
    vec_t vec;
    vec.size = size;
    vec.content = (float*)calloc(size , sizeof(float));
    if (tab != NULL) {
        for (size_t i = 0; i < size; i++) {
            vec.content[i] = tab[i];
        }
    }
    return vec;
}

void detruire(vec_t vec) {
    free(vec.content);
}

void afficher(vec_t a) {
    putc('[', stdout);
    for (size_t i = 0; i < a.size; i++) {
        printf("%g", a.content[i]);
        if (i < a.size - 1) {
            putc(' ', stdout);
        }
    }
    putc(']', stdout);
}

vec_t zip(float(*zipper)(float a, float b), vec_t a, vec_t b) {
    size_t rsize = (a.size < b.size ? a.size : b.size); // Minimum entre a et b
    vec_t resultat;
    resultat.size = rsize;
    resultat.content = (float*)malloc(rsize * sizeof(float));
    for (size_t i = 0; i < rsize; i++) {
        resultat.content[i] = zipper(a.content[i], b.content[i]);
    }
    return resultat;
}

float plus(float a, float b) {
    return a + b;
}

float moins(float a, float b) {
    return a - b;
}

float fois(float a, float b) {
    return a * b;
}

vec_t vec_plus(vec_t a, vec_t b) { zip(&plus, a, b); }
vec_t vec_moins(vec_t a, vec_t b) { zip(&moins, a, b); }
vec_t vec_fois(vec_t a, vec_t b) { zip(&fois, a, b); }

int main(void) {
    float ca[] = { 1.0, 2.0, 3.0 },
          cb[] = { 2.0, 4.0, 6.0 };

    vec_t a = creer(3, ca),
          b = creer(3, cb),
          r1, r2, r3;

    printf("a = ");
    afficher(a);
    printf(", b = ");
    afficher(b);
    puts("");

    r1 = vec_plus(a, b);
    r2 = vec_moins(a, b);
    r3 = vec_fois(a, b);

    printf("a + b = ");
    afficher(r1);
    puts("");

    printf("a - b = ");
    afficher(r2);
    puts("");

    printf("a × b = ");
    afficher(r3);
    puts("");

    detruire(a);
    detruire(b);
    detruire(r1);
    detruire(r2);
    detruire(r3);

    return 0;
}


a = [1 2 3], b = [2 4 6]
a + b = [3 6 9]
a - b = [-1 -2 -3]
a × b = [2 8 18]


À noter que : comme les pointeurs de fonction sont des pointeurs, ils peuvent valoir NULL. Tout comme on ne doit jamais déréférencer NULL, on ne doit jamais appeler NULL...

Le code devrait prendre toutes les mesures nécessaires pour s'assurer de ne pas appeler NULL.


In [3]:
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>

void bien(bool (*predicat)(int), int* tab, size_t taille) {
    for (size_t i = 0; i < taille; i++) {
        printf("tab[%ld]? ", i);
        if (predicat != NULL && predicat(tab[i])) {
            puts("OK");
        } else {
            puts("KO");
        }
    }
}

void pas_bien(bool (*predicat)(int), int* tab, size_t taille) {
    for (size_t i = 0; i < taille; i++) {
        printf("tab[%ld]? ", i);
        if (predicat(tab[i])) {
            puts("OK");
        } else {
            puts("KO");
        }
    }
}

bool est_pair(int x) { return (x % 2) == 0; }

int main(void) {
    int tab[] = { 1, 2, 3, 4, 5, 6, 7, 8 };

    puts("== bien est_pair ==");
    bien(&est_pair, tab, 8);
    puts("== bien NULL ==");
    bien(NULL, tab, 8);
    puts("== pas_bien est_pair ==");
    pas_bien(&est_pair, tab, 8);
    puts("== pas_bien NULL ==");
    pas_bien(NULL, tab, 8); // ça crash

    return 0;
}


== bien est_pair ==
tab[0]? KO
tab[1]? OK
tab[2]? KO
tab[3]? OK
tab[4]? KO
tab[5]? OK
tab[6]? KO
tab[7]? OK
== bien NULL ==
tab[0]? KO
tab[1]? KO
tab[2]? KO
tab[3]? KO
tab[4]? KO
tab[5]? KO
tab[6]? KO
tab[7]? KO
== pas_bien est_pair ==
tab[0]? KO
tab[1]? OK
tab[2]? KO
tab[3]? OK
tab[4]? KO
tab[5]? OK
tab[6]? KO
tab[7]? OK
== pas_bien NULL ==
tab[0]? 

[C kernel] Executable exited with code -11

### Fonctions comme champs d'enregistrement

Quand on a compris que les paramètres, les variables et les champs d'un enregistrement sont essentiellement la même chose, on réalise qu'on peut aussi stocker des fonctions dans un enregistrement. C'est particulièrement puissant, parce qu'on peut alors stocker au même endroit une fonction et des paramètres, par exemple, dans ce qui ressemble à ce qu'on appelle parfois une _fermeture_ (ou [_closure_](https://en.wikipedia.org/wiki/Closure_(computer_programming))).


In [8]:
//%ldflags: -lm
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

// Un layer réalise la fonction "f(x) = activation(coeff * x + offset)"
struct layer_t {
    double coeff;
    double offset;
    double(*activation)(double);
};
typedef struct layer_t layer_t;

double apply(layer_t layer, double x) {
    return layer.activation(layer.coeff * x + layer.offset);
}

double sigmoid(double x) {
    double ex = exp(x);
    return ex / (1.0 + ex);
}

int main() {
    layer_t layer1 = {
        .coeff = 0.5,
        .offset = 1.2,
        .activation = sigmoid
    };
    layer_t layer2 = {
        .coeff = 0.5,
        .offset = 1.2,
        .activation = tanh // fournis par math.h
    };

    for (double x = -10.0; x <= 10.0; x += 0.5) {
        printf("f1(%+ 6.2f) = %+ 10.6f   f2(%+ 6.2f) = %+ 10.6f\n", x, apply(layer1, x), x, apply(layer2, x));
    }

    return 0;
}

/tmp/tmp4lw69ncz.c: In function ‘main’:
/tmp/tmp4lw69ncz.c:36:16: warning: ' ' flag ignored with '+' flag in gnu_printf format [-Wformat=]
   36 |         printf("f1(%+ 6.2f) = %+ 10.6f   f2(%+ 6.2f) = %+ 10.6f\n", x, apply(layer1, x), x, apply(layer2, x));
      |                ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/tmp/tmp4lw69ncz.c:36:16: warning: ' ' flag ignored with '+' flag in gnu_printf format [-Wformat=]
/tmp/tmp4lw69ncz.c:36:16: warning: ' ' flag ignored with '+' flag in gnu_printf format [-Wformat=]
/tmp/tmp4lw69ncz.c:36:16: warning: ' ' flag ignored with '+' flag in gnu_printf format [-Wformat=]


f1(-10.00) =  +0.021881   f2(-10.00) =  -0.999000
f1( -9.50) =  +0.027923   f2( -9.50) =  -0.998351
f1( -9.00) =  +0.035571   f2( -9.00) =  -0.997283
f1( -8.50) =  +0.045217   f2( -8.50) =  -0.995524
f1( -8.00) =  +0.057324   f2( -8.00) =  -0.992632
f1( -7.50) =  +0.072426   f2( -7.50) =  -0.987880
f1( -7.00) =  +0.091123   f2( -7.00) =  -0.980096
f1( -6.50) =  +0.114052   f2( -6.50) =  -0.967395
f1( -6.00) =  +0.141851   f2( -6.00) =  -0.946806
f1( -5.50) =  +0.175086   f2( -5.50) =  -0.913785
f1( -5.00) =  +0.214165   f2( -5.00) =  -0.861723
f1( -4.50) =  +0.259225   f2( -4.50) =  -0.781806
f1( -4.00) =  +0.310026   f2( -4.00) =  -0.664037
f1( -3.50) =  +0.365864   f2( -3.50) =  -0.500520
f1( -3.00) =  +0.425557   f2( -3.00) =  -0.291313
f1( -2.50) =  +0.487503   f2( -2.50) =  -0.049958
f1( -2.00) =  +0.549834   f2( -2.00) =  +0.197375
f1( -1.50) =  +0.610639   f2( -1.50) =  +0.421899
f1( -1.00) =  +0.668188   f2( -1.00) =  +0.604368
f1( -0.50) =  +0.721115   f2( -0.50) =  +0.739783


### Types fonctions et `typedef`

Une dernière chose : il est possible de `typedef` un type pointeur de fonction. L'alias du type se trouve alors à l'endroit où se trouverait la variable déclarée avec ce type :

```c
// Définition du type "fonction_entier_vers_entier"
typedef int(*fonction_entier_vers_entier)(int);
```

Attention le type est bien celui d'un pointeur, donc ça s'utilise tel quel, sans `*` .


In [11]:
#include <stdio.h>
#include <stdlib.h>

typedef int(*fun_entier)(int);

void appliquer_afficher(fun_entier fun, int* tab, size_t taille) {
    for (size_t i = 0; i < taille; i++) {
        int r = fun(tab[i]);
        printf("resultat[%ld] = %d\n", i, r);
    }
}

int fois2(int x) { return x * 2; }
int plus1(int x) { return x + 1; }

int main() {
    int tab[] = { 2, 5, 3, 7, 1, 4 };

    puts("=== fois2 ===");
    appliquer_afficher(&fois2, tab, 6);
    puts("=== plus1 ===");
    appliquer_afficher(&plus1, tab, 6);

    return 0;
}

=== fois2 ===
resultat[0] = 4
resultat[1] = 10
resultat[2] = 6
resultat[3] = 14
resultat[4] = 2
resultat[5] = 8
=== plus1 ===
resultat[0] = 3
resultat[1] = 6
resultat[2] = 4
resultat[3] = 8
resultat[4] = 2
resultat[5] = 5


## Application au projet

Le projet Dijkstra est bien mais il manque un peu d'animation. En fait, la majeure partie du code est déjà présent pour animer la recherche du plus court chemin ; ce qu'il manque, c'est de l'intéraction entre votre algorithme de Dijkstra et l'interface graphique...

Idéalement, on aimerait que la fonction `dijkstra` cède le contrôle à des fonctions données par l'utilisateur à certaines étapes majeures ; l'utilisateur n'a alors plus qu'à définir les bonnes fonctions, qui sont appelées automatiquement par `dijkstra` : c'est un cas typique d'inversion de contrôle !

Voici ce qu'on propose : le module `dijkstra_event` se compose d'un unique header qui définit un enregistrement `struct dijkstra_event_t` , dont les champs sont chacun des fonctions que la fonction `dijkstra` devrait appeler au moment opportun (si la fonction n'est pas nulle, bien sûr). On appelle parfois de telles fonctions des _callback_ (des "rappels"), car elles redonnent la main à l'utilisateur.

L'idée est d'ajouter un argument de type `struct dijkstra_event_t` à la fonction `dijkstra` , et d'appeler ses champs dans l'implantation, lorsque c'est pertinent de le faire. Grossièrement :

```c
/* (dijkstra.h) */
...
// Nécessaire pour avoir accès au type struct dijkstra_event_t
#include "dijkstra_event.h"

...
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin,
    /* Les fonctions à appeler */
    struct dijkstra_event_t callback,
...
```

---

```c
/* (dijkstra.c) */
...
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin,
    /* Les fonctions à appeler */
    struct dijkstra_event_t callback,
...
    if (callback.on_visiting != NULL) { // Appel du callback à la visite du noeud.
        callback.on_visiting(noeud_courant, ...);
    }
...
```

Il reste cependant un mystère à élucider : toutes les fonctions de `struct dijkstra_event_t` ont pour dernier argument un argument de type `void*` , que la documentation appelle _user data_. De quoi s'agit-il, et que faire avec ?

<br>

En fait, il est possible que les fonctions aient besoin de certaines choses pour travailler, dont la fonction appelante n'est pas au courant (il faut considérer que l'appelant et les callback sont développées complètement séparément). Il est difficile de partager un état entre fonctions. Bien sûr, on pourrait utiliser une variable statique, mais c'est moche et dangereux.

En réalité, la seule façon sûre et correcte de partager un état, c'est d'utiliser un paramètre. Bien sûr, on ne connaît pas à l'avance le type de ce paramètre, car ça dépend complètement de la fonction donnée par l'utilisateur. Dans un langage bénéficiant de la généricité, on utiliserait un type générique (c'est ce que fait Java avec [sa version des callbacks](https://docs.oracle.com/javase/8/docs/api/java/util/function/Function.html)), mais on n'a pas ça en C, et à la place, on utilise un pointeur sans type stricte : `void*` .

Le principe est donc le suivant : les callback ont un paramètre `void*` (qu'on appelle généralement _user data_ : ce sont les données fournies par l'utilisateur) et l'appelant prend en paramètre non seulement les callback mais également un `void*` ; il a alors pour mission de transmettre les user data à chaque callback, pour qu'il puisse s'exécuter correctement.

```c
/* (dijkstra.h) */
...
// Nécessaire pour avoir accès au type struct dijkstra_event_t
#include "dijkstra_event.h"

...
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin,
    /* Les fonctions à appeler */
    struct dijkstra_event_t callback,
    /* Les données utilisateur à transmettre */
    void* userdata);
...
```

---

```c
/* (dijkstra.c) */
...
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin,
    /* Les fonctions à appeler */
    struct dijkstra_event_t callback,
    /* Les données utilisateur à transmettre */
    void* userdata) {
...
    if (callback.on_visiting != NULL) { // Appel du callback à la visite du noeud.
        callback.on_visiting(noeud_courant, userdata);
    }
...
```

Cette façon de faire est très commune en C. On pourrait l'appeler le "patron de conception callback", qui réalise donc une forme d'inversion de contrôle avec état persistant. C'est sur ce principe que repose par exemple les _signaux_ POSIX, les _threads_ et autres outils avancés du langage.


### Tâche à réaliser

Pour rendre interactif la fonction `dijkstra` écrite, il faut donc qu'elle accepte en paramètre un `struct dijkstra_event_t` et un `void*` . La fonction `dijkstra` a alors pour mission d'appeler les fonctions du `struct dijkstra_event_t` au bon moment lors de son déroulement :
 - `on_visiting` doit être appelée lorsque l'algorithme visite un noeud (à chaque tour de la boucle principale), avec comme paramètre le noeud en cours de visite (ce sont les noeuds en bleu sur l'animation)
 - `on_visited` doit être appelée lorsque l'algorithme a fini de visiter le noeud (à chaque fin de tour de la boucle principale), avec comme paramètre le noeud visité (ce sont les noeuds en vert sur l'animation)
 - `on_exploring` doit être appelée lorsque l'algorithme étudie un arc entre le noeud courant et un voisin (à chaque tour de boucle intérieure), avec comme paramètres le noeud courant et le noeud voisin en question (ce sont les arcs en bleu sur l'animation)
 - `on_marking` doit être appelée lorsque l'algorithme marque un noeud comme "à visitier", ou dit autrement lorsqu'il ajoute le noeud à la collection $AVisiter$, avec comme paramètre le noeud marqué (ce sont les noeuds en rouge sur l'animation)
 - `on_path_build` doit être appelée lors de la construction du chemin, à chaque étape de construction (à chaque fois qu'on ajoute un noeud), avec comme paramètre le chemin à l'étape courante

Pour pouvoir gérer la présence et l'absence d'animation dans le projet sans que ça pose problème, on a mis en place un mécanisme de pré-processeur. Le code de `main.c` par exemple, présente des section délimitées par `#ifdef AVEC_ANIMATION ... #endif` , qui ne sont activées que si la macro `AVEC_ANIMATION` a été définie.

Pour que tout se passe bien, votre code _doit aussi suivre ce principe_. Typiquement, la déclaration de `dijkstra` doit exister en deux versions : avec ou sans animation.

```c
#ifdef AVEC_ANIMATION
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin,
    /* Les fonctions à appeler */
    struct dijkstra_event_t callback,
    /* Les données utilisateur à transmettre */
    void* userdata);
#else // sans animation
// La version "normale"
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin);
#endif
```

Si vous vous sentez l'âme d'un vrai programmeur de C, on peut même factoriser un peu :

```c
float dijkstra(
    const struct graphe_t* graphe,
    noeud_id_t source, noeud_id_t destination,
    liste_noeud_t** chemin
#ifdef AVEC_ANIMATION
    , struct dijkstra_event_t callback,
    void* userdata
#endif
    );
```

De même, partout où vous faites référence au callback dans l'implantation, vous devriez isoler le code dans une section `#ifdef ... #endif` :

```c
...
#ifdef AVEC_ANIMATION
    if (callback.on_visiting != NULL) {
        callback.on_visting(noeud_courant, userdata);
    }
#endif
...
```

De cette façon, _votre projet continue de compiler avec le Makefile dans son état actuel !_ (et donc il n'y a pas de pénalité à la correction...).

<br>

Une fois que vous avez fait tout ça, on peut enfin compiler le projet _avec animation_. Pour cela, modifions un peu le Makefile : dans la variable `CFLAGS` , rajoutez `-DAVEC_ANIMATION` . Cela définit la macro `AVEC_ANIMATION` , ce qui devrait activer les sections qu'on a rajouté !

Recompilez le projet (il faut probablement le `clean` auparavant) et s'il n'y a pas d'erreur, lancez `./dijkstra` comme d'habitude : oh magie, il y a des couleurs partout !

Le fichier `main.c` contient les callback donné à `dijkstra` , que l'on peut modifier un peu. Typiquement, les fonctions changent des couleurs dans la représentation du graphe puis attendent un certain temps ( `SDL_Delay(200)` , donc 200 millisecondes a priori).


_Pour rappel, toute cette partie est **optionnelle**. Si vous la réalisez, elle pourrait entraîner un bonus, mais il faut surtout que votre projet compile et se lance sans encombre, sinon vous obtiendrez une note minimale..._

_Nous vous conseillons de faire un backup de votre projet avant de vous jeter dans cette partie, pour avoir une version qui marche sous la main en cas de problème._

_Bien sûr, cette partie n'est à réaliser que **si et seulement si votre projet marche déjà très bien**._
